<a href="https://colab.research.google.com/github/Ovocode05/NeuroDeff/blob/main/pybase_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NN from Scratch (Single Layer Perceptron) OR AND Logic Gates

In [1]:
import numpy as np

In [ ]:
class Neuron:
  def __init__(self, input_size):
    #xavier normalization for sigmoid function
    self.weights = np.random.normal(0, np.sqrt(2/(input_size+1)), input_size)
    self.bias = np.zeros(1)

  #functions
  def sigmoid(self, x):
    return 1/(1+np.exp(-x))

  def tanh(self, x):
    return np.tanh(x)

  def forward(self, x):
    # return self.sigmoid(np.dot(self.weights, x) + self.bias)
    return self.tanh(np.dot(self.weights, x) + self.bias)

  def MSE_derivative(self, y, y_pred):
    # return y_pred*(1-y_pred)*(y-y_pred) #Mse + sigmoid
    return (1 - y_pred**2) * (y - y_pred) #Mse + tanh

  #training
  def train(self, x, Y, lr=0.05, epochs=1000):
    for _ in range(epochs):
      total_loss= 0
      for xi, yi in zip(x,Y):
        y_pred = self.forward(xi) #range of y_pred [0,1]
        error = self.MSE_derivative(yi, y_pred)
        # error = yi-y_pred

        self.weights += lr*error*xi
        self.bias += lr*error

        self.weights = np.clip(self.weights, -10, 10)
        self.bias = np.clip(self.bias, -10, 10)
    return self.weights, self.bias


In [ ]:
X_or = np.array([[0,0], [0,1],[1,0],[1,1]])
Y_or = np.array([-1,1,1,1])
neuron = Neuron(input_size = 2)
print(neuron.train(X_or,Y_or))
print([1 if neuron.forward(x)> 0 else -1 for x in X_or])

#and gate logic
X_and = np.array([[0,0], [0,1],[1,0],[1,1]])
Y_and = np.array([-1,-1,-1,1])
neuron2 = Neuron(input_size = 2)
print(neuron2.train(X_and,Y_and))
print([1 if neuron2.forward(x)> 0 else -1 for x in X_and])

(array([2.96029645, 2.96142974]), array([-1.35546999]))
[-1, 1, 1, 1]
(array([2.62007346, 2.61824336]), array([-3.97228672]))
[-1, -1, -1, 1]


Multi Layer Perceptron (XOR)

In [24]:
class Neuron_Network:
  def __init__(self, input_size, hidden_size, output_size):
    # Xavier Normalization
    self.weights1 = np.random.normal(0, np.sqrt(2/(input_size + hidden_size)), (input_size, hidden_size))
    self.bias1 = np.zeros(hidden_size)
    self.weights2 = np.random.normal(0, np.sqrt(2/(hidden_size + output_size)), (hidden_size,output_size))
    self.bias2 = np.zeros(output_size)

  def sigmoid(self, x):
    return 1/(1+np.exp(-x))

  def sigmoid_derivative(self, x):
    return x*(1-x)

  def Relu(self, x):
    return np.maximum(0, x)

  def Relu_derivative(self, x):
    return np.where(x > 0, 1, 0)

  def forward(self, x):
    self.h_input = np.dot(x, self.weights1) + self.bias1 #hidden layer input
    self.h_output = self.Relu(self.h_input) #hidden layer output

    self.y_input = np.dot(self.h_output ,self.weights2) + self.bias2 #output layer input
    self.y_pred = self.sigmoid(self.y_input) #output layer output
    return self.y_pred

  def binary_CE(self,y, y_pred):
    return -np.sum(y*np.log(y_pred)+(1-y)*np.log(1-y_pred))

  def train(self, x, Y, lr=0.01, epoch=10000):
    for _ in range(epoch):
      y_pred = self.forward(x)
      error = y_pred - Y

      #output layer gradient
      d_output = error*self.sigmoid_derivative(y_pred)
      grad_w2 = np.dot(self.h_output.T, d_output)
      grad_b2 = np.sum(d_output, axis=0)

      #hidden layer gradient
      d_hidden = np.dot(d_output, self.weights2.T)*self.Relu_derivative(self.h_output)
      grad_w1 = np.dot(x.T, d_hidden)
      grad_b1 = np.sum(d_hidden, axis=0)

      #update weights
      self.weights1 -= lr*grad_w1
      self.bias1 -= lr*grad_b1
      self.weights2 -= lr*grad_w2
      self.bias2 -= lr*grad_b2

    return self.weights1, self.bias1, self.weights2, self.bias2
    #relu shows faster convergence, derivatives either 0 or 1
    #in case sigmoid weight updates are negligible during back propagation

In [25]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])
neuron3 = Neuron_Network(2,2,1)
neuron3.train(X,y)
# Test predictions
print("XOR Predictions:")
print([0 if neuron3.forward(x) < 0.5 else 1 for x in X])

XOR Predictions:
[0, 1, 1, 0]
